In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
sys.path.append("..")

import datetime
import pathlib

from collections import OrderedDict 

import numpy as np
import pandas as pd

In [3]:
# Pytorch
import torch
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.autograd import Variable

# Custom
from dutils import Experiment
from trainer import fit
import visualization as vis
from tcga_datasets import SiameseDataset

# Models
from tcga_networks import EmbeddingNet, SiameseNet
from losses import ContrastiveLoss

# Metrics
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_mutual_info_score as ANMI

In [4]:
def getTCGA(disease):
    path = "/srv/nas/mk2/projects/pan-cancer/TCGA_CCLE_GCP/TCGA/TCGA_{}_counts.tsv.gz"
    files = [path.format(d) for d in disease]
    return files


def readGCP(files, biotype='protein_coding', mean=True):
    """
    Paths to count matrices.
    """
    data_dict = {}
    for f in files:
        key = os.path.basename(f).split("_")[1]
        data = pd.read_csv(f, sep='\t', index_col=0)
        # transcript metadata
        meta = pd.DataFrame([row[:-1] for row in data.index.str.split("|")],
                            columns=['ENST', 'ENSG', 'OTTHUMG', 'OTTHUMT', 'GENE-NUM', 'GENE', 'BP', 'BIOTYPE'])
        meta = pd.MultiIndex.from_frame(meta)
        data.index = meta
        # subset transcripts
        data = data.xs(key=biotype, level='BIOTYPE')
        data = data.droplevel(['ENST', 'ENSG', 'OTTHUMG', 'OTTHUMT', 'GENE-NUM', 'BP'])
        # average gene expression of splice variants
        data = data.T
        if mean:
            data = data.groupby(by=data.columns, axis=1).mean()
        data_dict[key] = data
    return data_dict


def uq_norm(df, q=0.75):
    """
    Upper quartile normalization of GEX for samples.
    """
    quantiles = df.quantile(q=q, axis=1)
    norm = df.divide(quantiles, axis=0)
    return norm


def process_TCGA(disease=['BRCA', 'LUAD', 'KIRC', 'THCA', 'PRAD', 'SKCM']):
    base="/srv/nas/mk2/projects/pan-cancer/TCGA_CCLE_GCP"
    # get files
    tcga_files = getTCGA(disease)
    # read meta/data
    tcga_meta = pd.read_csv(os.path.join(base, "TCGA/TCGA_GDC_ID_MAP.tsv"), sep="\t")
    tcga_raw = readGCP(tcga_files, mean=True)
    # combine samples
    tcga_raw = pd.concat(tcga_raw.values())
    # Upper quartile normalization
    tcga_raw = uq_norm(tcga_raw)
    # log norm
    tcga = tcga_raw.transform(np.log1p)
    return tcga, tcga_meta

In [5]:
def generate_fsets(data, n_features, steps=5):
    r = np.linspace(start=1, stop=n_features, num=steps, dtype='int')
    idx = [np.random.choice(data.shape[1], size=i, replace=False) for i in r]
    return idx

In [6]:
def feature_training(train_data, train_labels, test_data, test_labels, feature_idx, embedding, exp_dir, cuda=True):
    # Meta data
    meta_data = {"n_features":[],
                 "model":[],
                 "ANMI":[]}
    # Params
    batch_size = 8
    kwargs = {'num_workers': 10, 'pin_memory': True} if cuda else {'num_workers': 10}
    
    # Feature Index
    for batch, feat in enumerate(feature_idx):
        print("Number features: {}\n".format(len(feat)))
        exp_data = {'feature_idx':feat}
        # Define data
        siamese_train_dataset = SiameseDataset(data=train_data.iloc[:,feat],
                                           labels=train_labels,
                                           train=True)
        siamese_test_dataset = SiameseDataset(data=test_data.iloc[:,feat],
                                          labels=test_labels,
                                          train=False)
        # Loaders
        siamese_train_loader = torch.utils.data.DataLoader(siamese_train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
        siamese_test_loader = torch.utils.data.DataLoader(siamese_test_dataset, batch_size=batch_size, shuffle=False, **kwargs)
        # Instantiate model
        n_samples, n_features = siamese_train_dataset.train_data.shape
        for i in range(3):
            nmodel = 'model_{}'.format(i)
            print("\t{}".format(nmodel))
            embedding_net = EmbeddingNet(n_features, embedding)
            model = SiameseNet(embedding_net)
            if cuda:
                model.cuda()
            # Parameters
            margin = 1.
            loss_fn = ContrastiveLoss(margin)
            lr = 1e-3
            optimizer = optim.Adam(model.parameters(), lr=lr)
            scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
            n_epochs = 10
            log_interval = round(len(siamese_train_dataset)/1/batch_size)
            # Train
            train_loss, val_loss = fit(siamese_train_loader, siamese_test_loader, model, loss_fn, optimizer, scheduler, 
                                       n_epochs, cuda, log_interval)
            # Test Embeddings
            val_embeddings_baseline, val_labels_baseline = vis.extract_embeddings(siamese_test_dataset.test_data, siamese_test_dataset.labels, model)
            # Evaluation
            n_clusters = len(np.unique(test_labels))
            kmeans = KMeans(n_clusters=n_clusters)
            siamese_clusters = kmeans.fit_predict(val_embeddings_baseline)
            anmi = ANMI(siamese_clusters, val_labels_baseline)
            # Store
            meta_data['n_features'].append(len(feat))
            meta_data['model'].append(nmodel)
            meta_data['ANMI'].append(anmi)
            exp_data[nmodel] = {'data': (val_embeddings_baseline, val_labels_baseline),
                                'loss': (train_loss, val_loss),
                                'ANMI': anmi}
        pd.to_pickle(exp_data, os.path.join(exp_dir, "model_{}.pkl".format(len(feat))))
    pd.to_pickle(meta_data, os.path.join(exp_dir, "model_meta_data.pkl"))

In [11]:
def main(disease, sample_type, **kwargs):
    # GPUs
    os.environ["CUDA_VISIBLE_DEVICES"] = kwargs['device']
    cuda = torch.cuda.is_available()
    print("Cuda is available: {}".format(cuda))
    
    # Read / write / process
    tcga, tcga_meta = process_TCGA(disease)
    # Feature design
    feature_idx = generate_fsets(tcga, n_features=kwargs['n_features'], steps=kwargs['steps'])
    # Experiment design
    hierarchy = OrderedDict({'Disease':disease,
                             'Sample Type':sample_type})
    # Define experiment
    exp = Experiment(meta_data=tcga_meta,
                     hierarchy=hierarchy,
                     index='CGHubAnalysisID',
                     cases='Case ID',
                     min_samples=20)
    # Train / Test split
    exp.train_test_split(cases='Case ID')
    # Return data 
    train_data, train_labels = exp.get_data(tcga, subset="train", dtype=np.float32)
    test_data, test_labels = exp.get_data(tcga, subset="test", dtype=np.float32)
    
    # randomize labels
    np.random.shuffle(train_labels)
    
    # Path *fix*
    dtime = datetime.datetime.today().strftime("%Y.%m.%d_%H:%M")
    exp_dir = "/srv/nas/mk2/projects/pan-cancer/experiments/feature_sel/{}_shuffle_{}_{}_{}-{}".format(dtime, 
                                                                                len(exp.labels_dict),
                                                                                kwargs['embedding'],
                                                                                kwargs['n_features'], 
                                                                                kwargs['steps'])
    pathlib.Path(exp_dir).mkdir(parents=True, exist_ok=False)
    print('Saving to: \n{}'.format(exp_dir))
    
    # Meta data
    experiments = {'experiment': exp,
                   'train':(train_data, train_labels),
                   'test': (test_data, test_labels)}
    pd.to_pickle(experiments, os.path.join(exp_dir, "experiment_meta_data.pkl"))
    
    # Training
    feature_training(train_data, train_labels, test_data, test_labels, feature_idx, kwargs['embedding'], exp_dir)

### Setup

In [9]:
disease = ['BRCA', 'LUAD', 'KIRC', 'THCA', 'PRAD', 'SKCM']
sample_type = ['Primary Tumor', 'Solid Tissue Normal']
params = {"device":"4",
          "n_features":2000,
          "steps":100,
          "embedding":2}

In [ ]:
main(disease=disease, sample_type=sample_type, **params)

Cuda is available: True
Saving to: 
/srv/nas/mk2/projects/pan-cancer/experiments/feature_sel/2020.03.27_18:57_shuffle_11_2_2000-100
Number features: 1

	model_0
Train: [0/2931 (0%)]	Loss: 0.187198
Train: [1098/2931 (100%)]	Loss: 0.178199
Epoch: 1/10. Train set: Average loss: 0.1782
Epoch: 1/10. Validation set: Average loss: 0.1560
Train: [0/2931 (0%)]	Loss: 0.165139
Train: [1098/2931 (100%)]	Loss: 0.176239
Epoch: 2/10. Train set: Average loss: 0.1762
Epoch: 2/10. Validation set: Average loss: 0.1452
Train: [0/2931 (0%)]	Loss: 0.183051
Train: [1098/2931 (100%)]	Loss: 0.175056
Epoch: 3/10. Train set: Average loss: 0.1751
Epoch: 3/10. Validation set: Average loss: 0.1372
Train: [0/2931 (0%)]	Loss: 0.206648
Train: [1098/2931 (100%)]	Loss: 0.174462
Epoch: 4/10. Train set: Average loss: 0.1746
Epoch: 4/10. Validation set: Average loss: 0.1371
Train: [0/2931 (0%)]	Loss: 0.113080
Train: [1098/2931 (100%)]	Loss: 0.174879
Epoch: 5/10. Train set: Average loss: 0.1747
Epoch: 5/10. Validation set: 